# Eat Safe, Love

## Notebook Set Up

In [295]:
import json
import pandas as pd
from pprint import pprint

# Ensure pymongo is installed
!pip install pymongo

from pymongo import MongoClient

# Load data from the JSON file with explicit encoding
with open('resources/establishments.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [296]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['uk_food']
establishments = db['establishments']

# Clear the collection to avoid duplicate entries (optional)
establishments.delete_many({})

# Insert the JSON data into MongoDB
if isinstance(data, list):
    establishments.insert_many(data)
else:
    establishments.insert_one(data)

# Verify insertion by printing a few documents
print("Sample documents:")
for doc in establishments.find().limit(5):
    pprint(doc)

Sample documents:
{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6654ea88c1e1823686903ca2'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDa

In [297]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [298]:
# Review the collections in our database
print("Collections in the uk_food database:")
print(db.list_collection_names())

Collections in the uk_food database:
['establishments']


In [299]:
# Assign the establishments collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [300]:
# Define the query to find establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}  # Adjust based on the actual field name and type

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments with a hygiene score of 20:", count)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
print("First document with a hygiene score of 20:")
pprint(first_document)

# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(list(establishments.find(query)))

# Print the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(result_df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(result_df.head(10))

Number of establishments with a hygiene score of 20: 41
First document with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6654ea88c1e1823686905abb'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/

In [301]:
import pandas as pd

# Execute the query and convert the result to a Pandas DataFrame
result = db.establishments.find(query)
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  6654ea88c1e1823686905abb   110681                  0   
1  6654ea88c1e1823686905e3c   612039                  0   
2  6654ea88c1e1823686906146   730933                  0   
3  6654ea88c1e1823686906334   172735                  0   
4  6654ea88c1e1823686906343   172953                  0   
5  6654ea88c1e1823686906ce2   512854                  0   
6  6654ea88c1e1823686906f03  1537089                  0   
7  6654ea88c1e182368690842e   155648                  0   
8  6654ea88c1e1823686908872  1012883                  0   
9  6654ea88c1e1823686909084   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [302]:
from pprint import pprint

# Define the query
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = db.establishments.count_documents(query)
print("Number of establishments with London as the Local Authority and RatingValue >= 4:", count)

# Display the first document in the results using pprint
first_document = db.establishments.find_one(query)
print("First document with London as the Local Authority and RatingValue >= 4:")
pprint(first_document)


Number of establishments with London as the Local Authority and RatingValue >= 4: 0
First document with London as the Local Authority and RatingValue >= 4:
None


In [303]:
import pandas as pd

# Execute the query and convert the result to a Pandas DataFrame
result = db.establishments.find(query)
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [304]:
degree_search = 0.01
latitude = 51.5074  # Example latitude for London
longitude = -0.1278  # Example longitude for London

# Define the query
query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

# Define sort criteria
sort = [("scores.Hygiene", 1)]  # Sort by hygiene score in ascending order

# Define limit
limit = 10  # Limit to 10 results

# Execute the query
result = db.establishments.find(query).sort(sort).limit(limit)

# Print the results
for doc in result:
    pprint(doc)

In [305]:
import pandas as pd

# Convert the query result to a list of dictionaries
result_list = list(result)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Print the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [306]:
# Define the pipeline stages
pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Match establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority and count
    {"$sort": {"count": -1}}  # Sort by count in descending order
]

# Execute the aggregation pipeline
result = db.establishments.aggregate(pipeline)

# Print the number of documents in the result
count = sum(1 for _ in result)
print("Number of documents in the result:", count)

# Execute the aggregation pipeline again to reset the cursor
result = db.establishments.aggregate(pipeline)

# Print the first 10 results
print("First 10 results:")
for i, doc in enumerate(result):
    if i < 10:
        print(doc)
    else:
        break


Number of documents in the result: 56
First 10 results:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [307]:
# Define the pipeline stages
pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Match establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority and count
    {"$sort": {"count": -1}}  # Sort by count in descending order
]

# Execute the aggregation pipeline
result = db.establishments.aggregate(pipeline)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 56
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
